In [ ]:
using JLD2
using Flux

In [ ]:
@load "Training.jld2" inputs;


In [ ]:
@load "Targets.jld2"  actuals;

In [ ]:
input = inputs[2,:];
input = reshape(input,:,100);
input[1,:]

In [ ]:
idx = rand(1:length(inputs[:,1]))
testSet = inputs[idx,:];
testSet = reshape(testSet,:,100);
testTarg = actuals[idx]

In [ ]:
actual = actuals[1];

In [ ]:
model = Chain(
    RNN(100,1, NNlib.relu)
) |> gpu;

In [ ]:
Flux.reset!(model)

In [ ]:
# out1=-1
# for i in eachrow(testSet)
#     out1 = model(i |> gpu) |> cpu
# end
loss(testSet, testTarg)

In [ ]:
Flux.reset!(model)
function eval_model(x)
    out1=-1
    for i in eachrow(x)
        out1 = model(i |> gpu) |> cpu
    end
    out1
end

loss(x, y) = Flux.Losses.mse(eval_model(x), y)

ps = Flux.params(model)

opt = Flux.ADAM();

evalcb() = @show(sum(loss.(testSet, testTarg)))

In [ ]:
num_epochs = length(inputs[:,1])
for x in 1:num_epochs
    Flux.train!(loss, ps, zip(inputs[x,:], actuals[x]), opt, cb = Flux.throttle(evalcb, 1))
end

In [ ]:
println("Test loss after = ", sum(loss.(test_data, test_labels)))